In [1]:

# imports
import sys
from utils_create import *
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import config
config = config.assessment_sources
SPREADSHEET_ID_SAT = config['spreadsheet_id_SAT']
SPREADSHEET_ID_NAEP = config['spreadsheet_id_NAEP']
SPREADSHEET_ID_LSAT = config['spreadsheet_id_LSAT']
SPREADSHEET_ID_GRE = config['spreadsheet_id_GRE']
SPREADSHEET_ID_GMAT = config['spreadsheet_id_GMAT']
SPREADSHEET_ID_Casper = config['spreadsheet_id_Casper']
SPREADSHEET_ID_AAMC = config['spreadsheet_id_AAMC']
SPREADSHEET_ID_Mapping = config['spreadsheet_id_Mapping']


In [2]:
# connect to google sheets and get dfs
service = connect_to_google('sheets')

# make a list of all the spreadsheet ids in config.assessment_sources
# for each spreadsheet id, get the data from the 'Data' tab
df_sat = read_google_sheet(service, SPREADSHEET_ID_SAT, 'Data')
df_naep = read_google_sheet(service, SPREADSHEET_ID_NAEP, 'Data')
df_lsat = read_google_sheet(service, SPREADSHEET_ID_LSAT, 'Data')
df_gre = read_google_sheet(service, SPREADSHEET_ID_GRE, 'Data')
df_gmat = read_google_sheet(service, SPREADSHEET_ID_GMAT, 'Data')
df_casper = read_google_sheet(service, SPREADSHEET_ID_Casper, 'Data')
df_aamc = read_google_sheet(service, SPREADSHEET_ID_AAMC, 'Data')
df_Mapping = read_google_sheet(service, SPREADSHEET_ID_Mapping, 'Variable')

In [3]:
# add N as last column to df_naep
df_naep['N'] = np.nan

# if a cell in df_mapping is empty put it as missing value
df_Mapping = df_Mapping.replace(r'^\s*$', np.nan, regex=True)

dfs_dict = {'df_sat': df_sat, 
            'df_naep': df_naep, 
            'df_lsat': df_lsat, 
            'df_gre': df_gre, 
            'df_gmat': df_gmat, 
            'df_casper': df_casper, 
            'df_aamc': df_aamc}

In [4]:
# create a mapping dictionay: 
mapping_table = df_Mapping
# Initialize a dictionary of dictionaries to store mappings
mappings = {}

# Iterate over each column in the mapping table (excluding 'CommonTerm' and 'Column')
for col in mapping_table.columns[2:]:
    # Create an empty dictionary for each data frame
    mappings[col] = {}
    
    # Filter the mapping table for each column type (e.g., 'Race', 'Gender')
    for column in mapping_table['Column'].unique():
        filtered_table = mapping_table[mapping_table['Column'] == column]
        # Create a sub-dictionary for each column in each data frame
        mappings[col][column] = dict(zip(filtered_table[col], filtered_table['CommonTerm']))


In [5]:
# Standardize relevant columns in each data frame using the mappings
for name, df in dfs_dict.items():
    for column in df.columns:
        if column in mappings[name]:  # Check if there's a mapping for this column
            df[column] = df[column].map(mappings[name][column]).fillna(df[column])


In [6]:
# Combine all the dataframes from the df_dict into one dataframe
combined_df = pd.concat(dfs_dict.values(), ignore_index=True)


In [7]:
# Create the dictionary
variable_grouping_dict = {}

# Iterate through each unique value in the 'Variable' column
for variable in combined_df['Variable'].unique():
	# Get the unique 'Grouping' values for the current 'Variable'
	groupings = combined_df[combined_df['Variable'] == variable]['Grouping'].unique()
	# Store in the dictionary
	variable_grouping_dict[variable] = list(groupings)

# Print the dictionary
print(variable_grouping_dict)

{'Home Language': ['Another Language', 'English', 'No Response'], 'Gender': ['Another gender', 'Female', 'Male', 'No Response'], 'Parent Education': ['Associate Degree', "Bachelor's Degree", 'Graduate Degree', 'High School Diploma', 'No High School Diploma', 'No Response'], 'Family Income': ['$75,000 to $99,999', '$50,000 to $74,999', '> $100,000', '< $50,000', 'No Response'], 'Race/Ethnicity': ['Indigenous/Native', 'Asian', 'Black', 'Hispanic', 'Native Hawaiian/Other Pacific Islander', 'No Response', 'Multiple Races/Ethnicities', 'White', 'Puerto Rican', 'Canadian Indigenous', 'Another Race/Ethnicity', 'Another Language', 'Middle East/North African American'], 'National School Lunch Program eligibility': ['Eligible', 'Not eligible', 'Information not available'], 'Receive Title I funding': ['No', 'Yes, for students', 'Yes, for school purp'], 'English Proficiency': ['Basic/Fair/Competent', 'Advanced/Functionally Native/Native', 'No Response'], 'UTOL4 – School location': ['City', 'Suburb

In [8]:
# clean up data

# set column types  for combined_df
column_types = {
    'Variable': 'category',
    'Subject': 'category',
    'Year': 'int64',
    'Jurisdiction': 'category',
    'Grouping': 'category',
    'Mean': 'float64',
    'SD': 'float64',
    'N': 'float64'
}

# Set the types of each column in combined_df using the dictionary
combined_df = combined_df.astype(column_types)

# Verify the column types
print(combined_df.dtypes)


Variable        category
Subject         category
Grade             object
Year               int64
Jurisdiction    category
Grouping        category
Mean             float64
SD               float64
N                float64
dtype: object


In [9]:
# for each df in dfs_dict, print  the unique Variable
for name, df in dfs_dict.items():
    print(f"{name} unique variables:")
    print(df['Variable'].unique())
    print()

df_sat unique variables:
['Home Language' 'Gender' 'Parent Education' 'Family Income'
 'Race/Ethnicity']

df_naep unique variables:
['Gender' 'National School Lunch Program eligibility' 'Parent Education'
 'Race/Ethnicity' 'Receive Title I funding' 'English Proficiency'
 'UTOL4 – School location']

df_lsat unique variables:
['Country' 'Gender' 'Race/Ethnicity' 'Region']

df_gre unique variables:
['Age Group' 'Citizenship' 'Gender' 'Race/Ethnicity'
 'Years of Full-time Work Experience']

df_gmat unique variables:
['Gender' 'Race/Ethnicity' 'Home Language' 'Citizenship' 'Age']

df_casper unique variables:
['Gender' 'Race/Ethnicity' 'Parent Education' 'Age' 'Citizenship'
 'Home Language' 'English Proficiency' 'Family Income']

df_aamc unique variables:
['Race/Ethnicity' 'Gender']



In [10]:
# get unique parent education from combined_df
combined_df['Grouping'].unique()

['Another Language', 'English', 'No Response', 'Another gender', 'Female', ..., 'Middle East/North African American', 'Under 20', '20-22', '26-28', 'Over 28']
Length: 74
Categories (74, object): ['$50,000 to $74,999', '$75,000 to $99,999', '1-2 Years', '11-14 Years', ..., 'Yes, for school purp', 'Yes, for students', '≤ 25', '≥ 31']

In [11]:
# Group by 'Variable' and collect the unique 'Grouping' values under each 'Variable'
grouped = combined_df.groupby('Variable')['Grouping'].apply(list).reset_index()

# Now, expand the lists to create a dataframe with 'Variable' as columns
expanded_df = pd.DataFrame({var: pd.Series(group) for var, group in grouped.set_index('Variable')['Grouping'].items()})

# Display the result
expanded_df

/var/folders/cc/rk1x9by94ks5m201ycvs1b0m0000gn/T/ipykernel_49631/644546536.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = combined_df.groupby('Variable')['Grouping'].apply(list).reset_index()


,Age,Age Group,Citizenship,Country,English Proficiency,Family Income,Gender,Home Language,National School Lunch Program eligibility,Parent Education,Race/Ethnicity,Receive Title I funding,Region,UTOL4 – School location,Years of Full-time Work Experience
0,≤ 25,18-22,International,US,Basic/Fair/Competent,"$75,000 to $99,999",Another gender,Another Language,Eligible,Associate Degree,Indigenous/Native,No,Northeast,City,No Response
1,26–30,18-22,International,Canada,Advanced/Functionally Native/Native,"$50,000 to $74,999",Female,English,Not eligible,Bachelor's Degree,Asian,"Yes, for students",Southeast,Suburb,No Response
2,≥ 31,18-22,International,NaN,Basic/Fair/Competent,"> $100,000",Male,English,Information not available,Graduate Degree,Black,"Yes, for school purp",Far West,Town,No Response
3,Under 20,23-25,Domestic,NaN,Advanced/Functionally Native/Native,"< $50,000",Another gender,No Response,Eligible,High School Diploma,Hispanic,No,Great Lakes,Rural,Less than 1 Year
4,20-22,23-25,Domestic,NaN,Basic/Fair/Competent,"$75,000 to $99,999",Female,Another Language,Not eligible,No High School Diploma,Native Hawaiian/Other Pacific Islander,"Yes, for students",Mid-South,City,Less than 1 Year
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Native Hawaiian/Other Pacific Islander,NaN,NaN,NaN,NaN
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,White,NaN,NaN,NaN,NaN
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Another Race/Ethnicity,NaN,NaN,NaN,NaN
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multiple Races/Ethnicities,NaN,NaN,NaN,NaN


# Cohen's D

In [12]:
# Reference groups dictionary
reference_groups = {
	'Gender': 'Female',
	'Race/Ethnicity': 'White',
 	'Family Income': '> $100,000',
	'Parent Education': 'Bachelor\'s Degree',
	'English Proficiency': 'Advanced/Functionally Native/Native',
	'Home Language': 'English',
	'Citizenship': 'Domestic',
	'National School Lunch Program eligibility': 'Not eligible',
}


# Example usage
# combined_df = pd.read_csv('path_to_your_csv_file.csv')  # Load your DataFrame
cohens_d_df = calculate_cohens_d(combined_df, reference_groups)
print(cohens_d_df)

           Variable      Subject  Year Jurisdiction Reference Group  \
0       Citizenship       Casper  2022           CA        Domestic   
1       Citizenship       Casper  2022           CA        Domestic   
2       Citizenship       Casper  2022           US        Domestic   
3       Citizenship       Casper  2022           US        Domestic   
4       Citizenship       Casper  2024           CA        Domestic   
..              ...          ...   ...          ...             ...   
440  Race/Ethnicity  SAT - Total  2023           US           White   
441  Race/Ethnicity  SAT - Total  2023           US           White   
442  Race/Ethnicity  SAT - Total  2023           US           White   
443  Race/Ethnicity  SAT - Total  2023           US           White   
444  Race/Ethnicity  SAT - Total  2023           US           White   

                           Comparison Group  Cohen's d  
0                               No Response  -0.167447  
1                             Int

/Users/alexandermacintosh/Documents/GitHub/prototypes/score-gaps/score-gaps-across-assessments/utils_create.py:90: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (variable, subject, year, jurisdiction) in combined_df.groupby(['Variable', 'Subject', 'Year', 'Jurisdiction']).groups.keys():


In [13]:
# Rename 'Comparison Group' to 'Grouping' in cohens_d_df to facilitate the merge
cohens_d_df = cohens_d_df.rename(columns={'Comparison Group': 'Grouping'})

# Keep only the 'Cohen's d' column in cohens_d_df
cohens_d_df = cohens_d_df[['Variable', 'Subject', 'Year', 'Jurisdiction', 'Grouping', "Cohen's d"]]

# Merge cohens_d_df with combined_df on the relevant columns
merged_df = combined_df.merge(cohens_d_df, on=['Variable', 'Subject', 'Year', 'Jurisdiction', 'Grouping'], how='left')

# Assign a Cohen's d value of 0 where the reference category matches the grouping
for variable, reference_group in reference_groups.items():
    merged_df.loc[(merged_df['Variable'] == variable) & (merged_df['Grouping'] == reference_group), "Cohen's d"] = 0

# Verify the result
print(merged_df)

          Variable                 Subject Grade  Year  Jurisdiction  \
0    Home Language               SAT - ERW    NA  2023            US   
1    Home Language               SAT - ERW    NA  2023            US   
2    Home Language               SAT - ERW    NA  2023            US   
3    Home Language               SAT - ERW    NA  2023            US   
4           Gender               SAT - ERW    NA  2023            US   
..             ...                     ...   ...   ...           ...   
792         Gender               MCAT PSBB    NA  2023  Matriculants   
793         Gender              MCAT Total    NA  2023  Matriculants   
794         Gender      AAMC - GPA Science    NA  2023  Matriculants   
795         Gender  AAMC - GPA Non-Science    NA  2023  Matriculants   
796         Gender        AAMC - GPA Total    NA  2023  Matriculants   

             Grouping    Mean      SD          N  Cohen's d  
0    Another Language  517.00  110.00   200762.0  -0.118182  
1          

In [14]:
# Convert year into category using .loc to avoid SettingWithCopyWarning
merged_df.loc[:, 'Year'] = merged_df['Year'].astype('Int64')

# Create the index
merged_df['Assessment'] = merged_df['Subject'] + ' - ' + merged_df['Jurisdiction'] + ' - ' + merged_df['Year'].astype(str)


In [15]:
# only where cohen's d is not null
merged_df = merged_df[merged_df['Cohen\'s d'].notnull()]

In [16]:
# write merged_df to a csv file in data folder
merged_df.to_csv('merged_data.csv', index=False)